# Search for clear Planet Images covering a GeoJSON-specified area, download, and tile into 224x224 squares

## Created by Julian Lee, April 8 2021

In [2]:
## Import Packages - if not installed, use conda or pip to install

import requests
import os
import utm
import rasterio as rio
import shapely.geometry
import datetime


from pathlib import Path
from dateutil.relativedelta import relativedelta
from itertools import product
from rasterio import windows
from rasterio.mask import mask, raster_geometry_mask
from rasterio.io import MemoryFile

In [3]:
## Setup API Authentication

os.environ['PL_API_KEY'] = 'ef1d2935cae84b3cb4e4cadd95a0779e'
PLANET_API_KEY = os.getenv('PL_API_KEY')
url = 'https://api.planet.com/data/v1/'

session = requests.Session()
session.auth = (PLANET_API_KEY, '')

#### User-defined parameters

In [4]:
## Tile width & height
tile_width, tile_height = 224, 224

## Define overlap of tile windows in px
width_overlap, height_overlap = 20, 20

## Relative directory path to save tiles
out_path = './images/Planet/'+ datetime.date.today().isoformat() + '/'

## Number of days in the past to search for clear imagery. Can increase if no results.
days_range = 28

## Minimum image clarity percentage, determined by Planet, to filter. Can lower if no results, but may give cloudy images. 
clear_threshold = 70

## GeoJSON Feature area of interest: replace with your own. GeoJSONs can be generated from drawings at https://www.geojson.io
geojson = {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-64.818726,-9.498443],[-64.806529,-9.501545],[-64.813893,-9.528857],[-64.828469,-9.541491],[-64.834682,-9.531959],[-64.821641,-9.511078],[-64.818726,-9.498443]]]}}

#### Setup search filters

In [13]:
item_types = ['PSScene3Band', 'PSScene4Band']
end = datetime.datetime.utcnow() # <-- get current day in UTC
start = datetime.datetime.utcnow() - relativedelta(days=days_range)  

fltr = {
    "type": "AndFilter",
    "config": [
        {
            "type": "DateRangeFilter", 
            "field_name":"acquired",
            "config": {
              "gt": start.isoformat("T") + "Z",
              "lte": end.isoformat("T") + "Z"
           }
        },
        {
           "type": "GeometryFilter",
           "field_name":"geometry",
           "config": geojson['geometry']
        },
        {
           "type":"RangeFilter",
           "field_name":"clear_percent",
           "config":{
              "gte": clear_threshold
            }
        },
        {
         "type":"PermissionFilter",
         "config": [
                "assets:download"
            ]
        }
    ]
}

#### Send search to Planet API using given filters and choose the result with best coverage of our AOI

In [14]:
res = session.post(url + 'quick-search', json = {'filter': fltr, 'item_types': item_types})
features = res.json()['features']
aoi = shapely.geometry.shape(geojson['geometry'])

best_feature = None
best_coverage = 0

for feature in features:
    intersection = shapely.geometry.shape(feature['geometry']).intersection(aoi).area 
    if intersection > best_coverage: 
        best_feature = feature
        best_coverage = intersection

#### Send activation request to Planet API for the best result - this prepares the image for download

In [22]:
order_url = 'https://api.planet.com/compute/ops/orders/v2'

json = {
    'name': 'order-' + datetime.datetime.utcnow().isoformat(), 
    'subscription_id': 544465,
    'products': [
        {
            'item_ids': [best_feature['id']],
            'item_type': best_feature['properties']['item_type'],
            'product_bundle': 'analytic'
        }
    ],
    'delivery': {
        'single_archive': True
    },
    'notifications': {
        'email': True
    },
    'order_type': 'partial',
    'tools': [
        {
            'clip': {
                'aoi': geojson['geometry']   
            }
        }
    ]
}

In [23]:
if best_feature:
    res = session.post(order_url, json=json)
    if res.status_code == 202:
    ## This should output 202 
        print('Search successful, activation request status: ', res.status_code)
    else:
        print('Search unsuccessful, error: ', res.text)
else:
    print('No search results, widen filters')

Search successful, activation request status:  202
